In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("sms-spam-classifier")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000/")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .getOrCreate()
)

In [3]:
df = spark.read.parquet("s3a://delta/")
df.show()

+-----+--------------------+--------------------+-------+
|label|                 sms|                 raw|is_spam|
+-----+--------------------+--------------------+-------+
|  ham|Dear i have reach...|Dear i have reach...|  false|
|  ham|Fighting with the...|Fighting with the...|  false|
|  ham|When can ü come out?|When can ü come out?|  false|
|  ham|Check with nuerol...|Check with nuerol...|  false|
|  ham|Lolnice. I went f...|Lolnice. I went f...|  false|
| spam|+123 Congratulati...|+123 Congratulati...|   true|
|  ham|No it's waiting i...|No it's waiting i...|  false|
|  ham|Maybe westshore o...|Maybe westshore o...|  false|
|  ham|You should know n...|You should know n...|  false|
|  ham|What's the signif...|What's the signif...|  false|
|  ham|Your opinion abou...|Your opinion abou...|  false|
|  ham|8 at the latest, ...|8 at the latest, ...|  false|
|  ham|Prabha..i'm soryd...|Prabha..i'm soryd...|  false|
|  ham|Lol ok your forgi...|Lol ok your forgi...|  false|
|  ham|No..jst

In [4]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.pyspark.ml.autolog()
mlflow.start_run()

<ActiveRun: >

In [5]:
from pyspark.ml.feature import (
    CountVectorizer, StringIndexer, VectorAssembler, Tokenizer, RegexTokenizer, StopWordsRemover)
from nltk.corpus import stopwords

stages = []
regexTokenizer = RegexTokenizer(inputCol="sms", outputCol="tokens", pattern="\\W+")
stages += [regexTokenizer]

STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
remover = StopWordsRemover(stopWords=STOPWORDS, inputCol="tokens", outputCol="real_tokens")
stages += [remover]

cv = CountVectorizer(inputCol="real_tokens", outputCol="token_features", minDF=2.0)#, vocabSize=3, minDF=2.0
stages += [cv]

indexer = StringIndexer(inputCol="label", outputCol="label_num")
stages += [indexer]

vecAssembler = VectorAssembler(inputCols=['token_features'], outputCol="features")
stages += [vecAssembler]

for stage in stages:
    print(stage)

RegexTokenizer_d14a9d749b3a
StopWordsRemover_c72f9866d529
CountVectorizer_2a332918acd4
StringIndexer_9ac9df49d71f
VectorAssembler_ad188eb41c06


In [6]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)
data = pipeline.fit(df).transform(df)

2023/03/07 21:52:04 WARNING mlflow.utils: Truncated the value of the key `StopWordsRemover.stopWords`. Truncated value: `['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing...`
2023/03/07 21:52:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead."
2023/03/07 21:52:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/loc

In [7]:
train, test = data.randomSplit([0.75, 0.25], seed = 42)

train_s  = train.where('is_spam').count()
train_ns = train.where('not is_spam').count()
test_s   = test.where('is_spam').count()
test_ns  = test.where('not is_spam').count()
print(f'''
      Train spam:  {train_s}
      Train ham:   {train_ns}
      Train ratio: {train_s / train_ns} (spam/ham)
      
      Test spam:  {test_s}
      Test ham:   {test_ns}
      Test ratio: {test_s / test_ns} (spam/ham)
      ''')


      Train spam:  578
      Train ham:   3624
      Train ratio: 0.15949227373068434 (spam/ham)
      
      Test spam:  169
      Test ham:   1203
      Test ratio: 0.1404821280133001 (spam/ham)
      


In [8]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", labelCol="label_num")
model = nb.fit(train)

2023/03/07 21:55:49 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('tokens', ArrayType(StringType(), True), True), StructField('real_tokens', ArrayType(StringType(), True), True), StructField('token_features', VectorUDT(), True), StructField('features', VectorUDT(), True)]. Model signature is not logged.


In [9]:
predictions = model.transform(test)
predictions.select("label_num", "prediction", "probability").show()

+---------+----------+--------------------+
|label_num|prediction|         probability|
+---------+----------+--------------------+
|      0.0|       0.0|[0.99261568389786...|
|      0.0|       0.0|[0.99999976600938...|
|      0.0|       0.0|[0.66818703921423...|
|      0.0|       0.0|[0.99944368067061...|
|      0.0|       0.0|[0.99070125131954...|
|      0.0|       0.0|[0.99902425557805...|
|      0.0|       0.0|[0.98139506908352...|
|      0.0|       0.0|[0.94580670071342...|
|      0.0|       0.0|[0.99909646763770...|
|      0.0|       0.0|[0.99997739056063...|
|      0.0|       0.0|[0.99996787015427...|
|      0.0|       0.0|[0.99999999759138...|
|      0.0|       0.0|[0.99943190904288...|
|      0.0|       0.0|[0.99996803773852...|
|      0.0|       0.0|[0.99981908729548...|
|      0.0|       0.0|[0.98441744031794...|
|      0.0|       0.0|[0.96931889397476...|
|      0.0|       0.0|[0.78895353392514...|
|      0.0|       0.0|[0.99999891154734...|
|      0.0|       0.0|[0.9999102

In [10]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label_num")
accuracy = evaluator.evaluate(predictions)
print ("Test Area Under ROC: ", accuracy)

Test Area Under ROC:  0.9760141067449719


In [ ]:
model_info = mlflow.spark.log_model(model, "naive-bayes")
print(model_info)

In [ ]:
mlflow.end_run()